## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':5,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [5]:
train = pd.read_csv('C:/Users/?/Desktop/Research/Projects/CP2/open/train.csv')
test = pd.read_csv('C:/Users/?/Desktop/Research/Projects/CP2/open/test.csv')

In [6]:
# EDA
def data_preprocessing(dataframe):
    
    # drop_duplicates
    dataframe.drop_duplicates(inplace=True)
    
    # drop non-training columns
    dataframe.drop(['DCIS_or_LCIS_type', 'HER2_SISH', 'HER2_SISH_ratio'], axis = 1, inplace=True)
    
    # fillna BRCA_mutation, 3 => '3' is new category that means 'not examine'
    dataframe['BRCA_mutation'].fillna(3.0, inplace=True)
    
    # data processing about 'HG' columns
    for i in range(len(dataframe)):
        # if every values about 'HG' is null that fill in grade '4'
        if (np.isnan(dataframe.loc[i, 'HG'])) & (np.isnan(dataframe.loc[i, 'HG_score_1'])) & (np.isnan(dataframe.loc[i, 'HG_score_2'])) & (np.isnan(dataframe.loc[i, 'HG_score_3'])):
            dataframe.loc[i, 'HG'] = 4.0
            dataframe.loc[i, 'HG_score_1'] = 4.0
            dataframe.loc[i, 'HG_score_2'] = 4.0
            dataframe.loc[i, 'HG_score_3'] = 4.0
        # if every values about 'HG_score' except 'HG' is null that fill in condition
        elif (not np.isnan(dataframe.loc[i, 'HG'])) & (np.isnan(dataframe.loc[i, 'HG_score_1'])) & (np.isnan(dataframe.loc[i, 'HG_score_2'])) & (np.isnan(dataframe.loc[i, 'HG_score_3'])):
            # if 'HG' is 2.0 that fill in 2.0
            if dataframe.loc[i, 'HG'] == 2.0:
                dataframe.loc[i, 'HG_score_1'] = 2.0
                dataframe.loc[i, 'HG_score_2'] = 2.0
                dataframe.loc[i, 'HG_score_3'] = 2.0
            # if 'HG' is 3.0 that fill in 3.0
            elif dataframe.loc[i, 'HG'] == 3.0:
                dataframe.loc[i, 'HG_score_1'] = 3.0
                dataframe.loc[i, 'HG_score_2'] = 3.0
                dataframe.loc[i, 'HG_score_3'] = 3.0
            else:
                dataframe.loc[i, 'HG_score_1'] = 1.0
                dataframe.loc[i, 'HG_score_2'] = 1.0
                dataframe.loc[i, 'HG_score_3'] = 1.0
        # other 'HG' cases drop index at the end of EDA
    
    # data processing about 'NG' columns
    for f in range(len(dataframe)):
        if (np.isnan(dataframe.loc[f, 'NG'])) & (dataframe.loc[f, 'HG'] == 1.0) | (dataframe.loc[f, 'HG'] == 4.0):
            dataframe.loc[f, 'NG'] = 1.0
        elif (np.isnan(dataframe.loc[f, 'NG'])) & (dataframe.loc[f, 'HG'] == 2.0):
            dataframe.loc[f, 'NG'] = 2.0
        elif (np.isnan(dataframe.loc[f, 'NG'])) & (dataframe.loc[f, 'HG'] == 3.0):
            dataframe.loc[f, 'NG'] = 3.0
    
    # 'T_category' column fill in mode
    dataframe['T_category'].fillna(dataframe['T_category'].mode()[0], inplace=True)
    
    # 'KI-67' column fill in mean
    dataframe['KI-67_LI_percent'].fillna(dataframe['KI-67_LI_percent'].mean(), inplace=True)
    
    # if 'ER' or 'PR' is null that fill in condition
    for g in range(len(dataframe)):
        if (np.isnan(dataframe.loc[g, 'ER'])) | (np.isnan(dataframe.loc[g, 'PR'])):
            dataframe.drop(g, inplace=True, axis = 0)
    dataframe.reset_index(drop=True, inplace=True)

    for j in range(len(dataframe)):
        if (dataframe.loc[j, 'ER'] == 0.0) & (np.isnan(dataframe.loc[j, 'ER_Allred_score'])):
            dataframe.loc[j, 'ER_Allred_score'] = 0.0
        elif (dataframe.loc[j, 'ER'] == 1.0) & (np.isnan(dataframe.loc[j, 'ER_Allred_score'])):
            dataframe.loc[j, 'ER_Allred_score'] = dataframe['ER'].mean()

    for d in range(len(dataframe)):
        if (dataframe.loc[d, 'PR'] == 0.0) & (np.isnan(dataframe.loc[d, 'PR_Allred_score'])):
            dataframe.loc[d, 'PR_Allred_score'] = 0.0
        elif (dataframe.loc[d, 'PR'] == 1.0) & (np.isnan(dataframe.loc[d, 'PR_Allred_score'])):
            dataframe.loc[d, 'PR_Allred_score'] = dataframe['PR'].mean()
    
    # '암의 장경' fill in mean
    dataframe['암의 장경'].fillna(dataframe['암의 장경'].mean(), inplace=True)

    # 'HER2_IHC' fill in contion
    for h in range(len(dataframe)):
        if np.isnan(dataframe.loc[h, 'HER2_IHC']):
            dataframe.loc[h, 'HER2_IHC'] = dataframe.loc[h, 'HER2']
        elif (dataframe.loc[h, 'HER2_IHC'] < 2) & (dataframe.loc[h, 'HER2'] == 1):
            dataframe.loc[h, 'HER2_IHC'] = 2.0
        elif (dataframe.loc[h, 'HER2_IHC'] > 1) & (dataframe.loc[h, 'HER2'] == 0):
            dataframe.loc[h, 'HER2_IHC'] = 1.0
        elif np.isnan(dataframe.loc[h, 'HER2']):
            if dataframe.loc[h, 'HER2_IHC'] >= 2:
                dataframe.loc[h, 'HER2'] = 1.0
            elif dataframe.loc[h, 'HER2_IHC'] < 2:
                dataframe.loc[h, 'HER2'] = 0.0
    
    # drop another 'NaN' values
    dataframe.dropna(inplace = True)
    dataframe.reset_index(drop=True, inplace=True)

    print(len(dataframe))
    print(dataframe.isnull().sum())
    return dataframe

In [7]:
train_set = data_preprocessing(train)
test_df = data_preprocessing(test)

996
ID                  0
img_path            0
mask_path           0
나이                  0
수술연월일               0
진단명                 0
암의 위치               0
암의 개수               0
암의 장경               0
NG                  0
HG                  0
HG_score_1          0
HG_score_2          0
HG_score_3          0
DCIS_or_LCIS_여부     0
T_category          0
ER                  0
ER_Allred_score     0
PR                  0
PR_Allred_score     0
KI-67_LI_percent    0
HER2                0
HER2_IHC            0
BRCA_mutation       0
N_category          0
dtype: int64
250
ID                  0
img_path            0
나이                  0
수술연월일               0
진단명                 0
암의 위치               0
암의 개수               0
암의 장경               0
NG                  0
HG                  0
HG_score_1          0
HG_score_2          0
HG_score_3          0
DCIS_or_LCIS_여부     0
T_category          0
ER                  0
ER_Allred_score     0
PR                  0
PR_Allred_score     0
KI-67_LI_pe

In [ ]:
train_set['img_path'] = train_set['img_path'].str.replace('\./train_imgs', 'C:/Users/?/Desktop/Research/Projects/CP2/open/train_imgs')
test_df['img_path'] = test_df['img_path'].str.replace('\./test_imgs', 'C:/Users/?/Desktop/Research/Projects/CP2/open/test_imgs')

train_set['img_path'].head()

In [11]:
train_df, val_df, train_labels, val_labels = train_test_split(
                                                    train_set.drop(columns=['N_category']), 
                                                    train_set['N_category'], 
                                                    test_size=0.2, 
                                                    random_state=CFG['SEED']
                                                )

In [14]:
print(len(train_df))
print(len(val_df))
print(len(train_labels))
print(len(val_labels))

796
200
796
200


In [15]:
def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent']
ignore_cols = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler()
        train_df[col] = scaler.fit_transform(get_values(train_df[col]))
        val_df[col] = scaler.transform(get_values(val_df[col]))
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        train_df[col] = le.fit_transform(get_values(train_df[col]))
        val_df[col] = le.transform(get_values(val_df[col]))
        test_df[col] = le.transform(get_values(test_df[col]))

## CustomDataset

In [16]:
class CustomDataset(Dataset):
    def __init__(self, medical_df, labels, transforms=None):
        self.medical_df = medical_df
        self.transforms = transforms
        self.labels = labels
        
    def __getitem__(self, index):
        img_path = self.medical_df['img_path'].iloc[index]
        image2 = cv2.imread(img_path)
        image = cv2.cvtColor(image2, cv2.COLOR_BGR2HSV)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
                
        if self.labels is not None:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', 'mask_path', '수술연월일']).iloc[index])
            label = self.labels[index]
            return image, tabular, label
        else:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', '수술연월일']).iloc[index])
            return image, tabular
        
    def __len__(self):
        return len(self.medical_df)

In [17]:
train_transforms = A.Compose([
                            A.HorizontalFlip(),
                            A.VerticalFlip(),
                            A.Rotate(limit=90, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transforms = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [18]:
train_dataset = CustomDataset(train_df, train_labels.values, train_transforms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df, val_labels.values, test_transforms)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Architecture

In [25]:
class ImgFeatureExtractor(nn.Module):
    def __init__(self):
        super(ImgFeatureExtractor, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.embedding = nn.Linear(1000, 512)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.embedding(x)
        return x

In [26]:
class TabularFeatureExtractor(nn.Module):
    def __init__(self):
        super(TabularFeatureExtractor, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(in_features=20, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=512),
        )
        
    def forward(self, x):
        x = self.embedding(x)
        return x

In [27]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.img_feature_extractor = ImgFeatureExtractor()
        self.tabular_feature_extractor = TabularFeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1),
            nn.Sigmoid(),
        )
        
    def forward(self, img, tabular):
        img_feature = self.img_feature_extractor(img)
        tabular_feature = self.tabular_feature_extractor(tabular)
        feature = torch.cat([img_feature, tabular_feature], dim=-1)
        output = self.classifier(feature)
        return output

## Train

In [28]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for img, tabular, label in tqdm(iter(train_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss, val_score = validation(model, criterion, val_loader, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
        
        if best_score < val_score:
            best_score = val_score
            best_model = model
    
    return best_model

In [29]:
def validation(model, criterion, val_loader, device):
    model.eval()
    pred_labels = []
    true_labels = []
    val_loss = []
    threshold = 0.5
    with torch.no_grad():
        for img, tabular, label in tqdm(iter(val_loader)):
            true_labels += label.tolist()
            
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            val_loss.append(loss.item())
            
            model_pred = model_pred.squeeze(1).to('cpu')  
            pred_labels += model_pred.tolist()
    
    pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
    val_score = metrics.f1_score(y_true=true_labels, y_pred=pred_labels, average='macro')
    return np.mean(val_loss), val_score

## Train

In [30]:
model = nn.DataParallel(ClassificationModel())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 13/13 [01:37<00:00,  7.48s/it]


Epoch [1], Train Loss : [0.64517] Val Loss : [0.62936] Val Score : [0.76611]


100%|██████████| 13/13 [01:37<00:00,  7.47s/it]


Epoch [2], Train Loss : [0.60586] Val Loss : [0.61792] Val Score : [0.74725]


100%|██████████| 13/13 [01:40<00:00,  7.71s/it]


Epoch [3], Train Loss : [0.58875] Val Loss : [0.61690] Val Score : [0.76881]


100%|██████████| 13/13 [01:38<00:00,  7.55s/it]


Epoch [4], Train Loss : [0.59159] Val Loss : [0.61460] Val Score : [0.78125]


100%|██████████| 13/13 [01:37<00:00,  7.51s/it]

Epoch [5], Train Loss : [0.57846] Val Loss : [0.61773] Val Score : [0.77494]


## Inference

In [31]:
test_dataset = CustomDataset(test_df, None, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [32]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    threshold = 0.5
    
    with torch.no_grad():
        for img, tabular in tqdm(iter(test_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            
            model_pred = model(img, tabular)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            
            preds += model_pred.tolist()
    
    preds = np.where(np.array(preds) > threshold, 1, 0)
    
    return preds

In [33]:
preds = inference(infer_model, test_loader, device)

100%|██████████| 16/16 [02:03<00:00,  7.72s/it]
